##Импорт библиотек

In [7]:
! pip install pytelegrambotapi

In [8]:
import torch
from PIL import Image
from torchvision import transforms
import numpy as np
import pandas as pd

In [9]:
import telebot
import requests
import shutil

##Иницилизация обвязки модели

In [10]:
resnet_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [11]:
! wget "https://getfile.dokpub.com/yandex/get/https://yadi.sk/d/DwxCSAMhhNtp9Q" -O mushroom_database.csv

--2021-10-29 08:07:25--  https://getfile.dokpub.com/yandex/get/https://yadi.sk/d/DwxCSAMhhNtp9Q
Resolving getfile.dokpub.com (getfile.dokpub.com)... 78.46.92.107
Connecting to getfile.dokpub.com (getfile.dokpub.com)|78.46.92.107|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://downloader.disk.yandex.ru/disk/e718bab48a73d3ee303e177cfec09dbb8972630b76294131dc7bba534cbd7a85/617be3fd/EnbAvYTdHMz2-Pr1wQCZdlsZY2tngKvNB_tdPZIhu6m2H9oQEErpjtwDwpf-3Pe0vHov-Zh1McUhjvKUxEm7lg%3D%3D?uid=0&filename=mushroom_database.csv&disposition=attachment&hash=JrUyajN%2ByEVY9ww5d/0LIVH10ncCGzCUJBKKj1E8DCnGL7rfaRCkmtLrm6cmYPY6q/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit=0&content_type=text%2Fplain&owner_uid=542830332&fsize=56941&hid=32addc8fe5ef899fedb03ff0ac3365c9&media_type=spreadsheet&tknv=v2 [following]
--2021-10-29 08:07:25--  https://downloader.disk.yandex.ru/disk/e718bab48a73d3ee303e177cfec09dbb8972630b76294131dc7bba534cbd7a85/617be3fd/EnbAvYTdHMz2-Pr1wQCZdlsZY2t

In [12]:
! wget "https://getfile.dokpub.com/yandex/get/https://yadi.sk/d/L1vdaKCEG2ZU-w" -O mushroom_model.pt

--2021-10-29 08:07:28--  https://getfile.dokpub.com/yandex/get/https://yadi.sk/d/L1vdaKCEG2ZU-w
Resolving getfile.dokpub.com (getfile.dokpub.com)... 78.46.92.107
Connecting to getfile.dokpub.com (getfile.dokpub.com)|78.46.92.107|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://downloader.disk.yandex.ru/disk/752629ba188a0adf29536fda65adafe375ee70dbf3d41bba28d420b4edcf32b5/617be401/EnbAvYTdHMz2-Pr1wQCZdvz6CH6pXczG5QcGx8zW6nwqNBTgjWXGKSseifeRVfH8g18mZPcrLqErdjuarKuX0Q%3D%3D?uid=0&filename=mushroom_model.pt&disposition=attachment&hash=aZU23jpjhOWHhUFGsyp9epyCf6ANnTmwRrMUl2aXjBu34bx%2BZpcoF7Rf2Sziuozmq/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit=0&content_type=application%2Fzip&owner_uid=542830332&fsize=44819917&hid=0a53b56ff4f1fdcf066ac84cb570588e&media_type=compressed&tknv=v2 [following]
--2021-10-29 08:07:29--  https://downloader.disk.yandex.ru/disk/752629ba188a0adf29536fda65adafe375ee70dbf3d41bba28d420b4edcf32b5/617be401/EnbAvYTdHMz2-Pr1wQCZdvz6

In [13]:
model_new = torch.load('mushroom_model.pt')

In [14]:
mtd = pd.read_csv('mushroom_database.csv')

##Запуск бота

In [41]:
API_TOKEN = '2027085491:AAGM3mxz4HHOn23AyCowB_A5uj2tMYU6i-0'
bot = telebot.TeleBot(API_TOKEN)

@bot.message_handler(content_types=["photo"])
def echo_message(message):
    file_info = bot.get_file(message.photo[-1].file_id) #const
    # bot.reply_to(message, f'Got your image: {file_info}')

    url = 'https://api.telegram.org/file/bot{0}/{1}'.format(API_TOKEN, file_info.file_path)
    response = requests.get(url, stream=True)
    
    path = 'image.jpg'
    if response.status_code == 200:
        with open(path, 'wb') as f:
            response.raw.decode_content = True
            shutil.copyfileobj(response.raw, f)
    
    img_test = Image.open(path)
    img_tensor = resnet_transforms(img_test)
    img_tensor = img_tensor.view(1, 3, 224, 224)
    output = model_new(img_tensor)
    prediction_new = int(torch.max(output.data, 1)[1].numpy())

    mushroom = mtd['mushroom'].values[prediction_new] + "\n" + "\n"
    description = mtd['description'].values[prediction_new] + "\n" + "\n"
    usefulness = mtd['usefulness'].values[prediction_new] + "\n" + "\n"
    fact = mtd['fact'].values[prediction_new] + "\n" + "\n"
    recipe = mtd['recipe'].values[prediction_new].replace(r'\n', '\n')

    bot.reply_to(message, mushroom + description + usefulness + fact + recipe)

In [ ]:
bot.infinity_polling()